In [98]:
from gurobipy import *
import numpy as np
import pandas as pd
from Hourbid import *
from Fibid import *
from blockbid import *


In [99]:
BS_bids = Blockbid()[0]
BD_bids = Blockbid()[1]
HS_bids = HourBid()[0]
HD_bids = HourBid()[1]
FS_bids = Fibid()[0]
FD_bids = Fibid()[1]

In [100]:
L_D = len(BD_bids)
L_S = len(Blockbid()[0])

In [101]:
Delta_S = np.zeros([L_S,24])


Delta_D = np.zeros([L_D,24])
BD_bids  = Blockbid()[1]
for i in range (L_D):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=25:
        Delta_D[i,BD_bids['Hour'][i]-1:(BD_bids['Hour'][i]+BD_bids['Duration'][i])-1] = 1

BS_bids = Blockbid()[0]
for i in range (L_S):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=25:
        Delta_S[i,BS_bids['Hour'][i]-1:(BS_bids['Hour'][i]+BS_bids['Duration'][i])-1] = 1

In [102]:
for i in range(L_D):
    if BD_bids['Hour'][i]==19:
        print(i)

19


In [103]:
BD_bids

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,1110,1,2,B,694.0,599.67,20,NaN
1,1106,1,2,B,562.0,553.92,11,NaN
2,1105,1,15,B,347.0,609.00,4,NaN
3,1102,1,15,B,543.0,581.19,8,NaN
4,1109,1,20,B,808.0,685.34,4,NaN
...,...,...,...,...,...,...,...,...
80,1078,1,2,B,129.0,716.49,19,NaN
81,1199,1,3,B,850.0,444.24,21,NaN
82,1090,1,11,B,172.0,563.73,10,NaN
83,1098,1,6,B,921.0,648.55,19,NaN


In [104]:
Delta_D[0,:]

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0.])

In [105]:
def find_F(Del,leng,bids):
    Res = [[] for i in range(24)]
    Res_max = [[] for i in range(24)]
    Res_min = [[] for i in range(24)]
    for i in range (24):
        for j in range(leng):
            if Del[j,i] == 1:
                Res[i].append(bids['price'][j])
    for i in range(24):
        Res_max[i] = np.max(Res[i])
        Res_min[i] = np.min(Res[i])
    return(Res_max,Res_min)

FD_min = np.min(FD_bids['price'])
FD_max = np.max(FD_bids['price'])
FS_min = np.min(FS_bids['price'])
FS_max = np.max(FS_bids['price'])

In [106]:
DMax_Res = [[] for i in range(24)]
Dm_Res = [[] for i in range(24)]
SMax_Res = [[] for i in range(24)]
Sm_Res = [[] for i in range(24)]

for i in range(24):
    DMax_Res[i].append(HD_bids['price'][HIndex_D[i][1]])
    DMax_Res[i].append(find_F(Delta_D,L_D,BD_bids)[0][i])
    DMax_Res[i].append(FD_max)
    
for i in range(24):
    SMax_Res[i].append(HS_bids['price'][HIndex_S[i][1]])
    SMax_Res[i].append(find_F(Delta_S,L_S,BS_bids)[0][i])
    SMax_Res[i].append(FS_max)
    
for i in range(24):
    Dm_Res[i].append(HD_bids['price'][HIndex_D[i][0]])
    Dm_Res[i].append(find_F(Delta_D,L_D,BD_bids)[1][i])
    Dm_Res[i].append(FD_min)
    
for i in range(24):
    Sm_Res[i].append(HS_bids['price'][HIndex_S[i][0]])
    Sm_Res[i].append(find_F(Delta_S,L_S,BS_bids)[1][i])
    Sm_Res[i].append(FS_min)

In [107]:
def findmin(num1,num2,num3):
    
    return(min([num1,num2,num3]))

def findmax(num1,num2,num3):

    return(max([num1,num2,num3]))


def put_into_list(maximums,minimums):
    F_min = []
    F_max = []
    for x in range(24):
        F_min.append(findmin(minimums[x][0],minimums[x][1],minimums[x][2]))
        F_max.append(findmax(maximums[x][0],maximums[x][1],maximums[x][2]))

    return(F_min,F_max)

In [108]:
E = Model("Energy")
E.params.NonConvex =2
E.update()
J = np.arange(1,find_largest(HIndex_D)+1)
K = np.arange(1,find_largest(HIndex_S)+1)
H = np.arange(1,25)

Set parameter NonConvex to value 2


In [109]:
L_FD = len(Fibid()[1])
L_FS = len(Fibid()[0])
L_FS

29

In [110]:
xD = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xD')
xS = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xS')
w_D = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_D')
w_S = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_S')
yD = E.addVars(L_D,ub=1,lb=0,  vtype=GRB.BINARY,name ='yD')
yS = E.addVars(L_S,ub=1,lb=0,  vtype=GRB.BINARY,name ='yS')
F_zS = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_ZS')
F_zD = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_ZD')
V_S = E.addVars(H,L_FS,lb=0,vtype=GRB.BINARY,name = 'V_S')
V_D = E.addVars(H,L_FD,lb=0,vtype=GRB.BINARY,name = 'V_D')

In [111]:
Data_H_D = HourBid()[1]
Data_H_SP = HourBid()[0]
FBid_S = Fibid()[0]
FBid_D = Fibid()[1]

In [112]:
FBid_S['Quantity'][1]

933.0

In [113]:
E.setObjective(sum(0.5*(2*Pair(j,h,Data_H_D,'DP')[0]+xD[h,j]*(Pair(j,h,Data_H_D,'DP')[1]-Pair(j,h,Data_H_D,'DP')[0]))*xD[h,j]*(Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0])for j in J for h in H)
                -sum(0.5*(2*Pair(j,h,Data_H_SP,'SP')[0]+xS[h,j]*(Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]))*xS[h,j]*(Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])for j in J for h in H)
                +sum(BD_bids['Duration'][i]*BD_bids['Quantity'][i]*BD_bids['price'][i]*yD[i] for i in range(L_D)) - sum(BS_bids['Duration'][i]*BS_bids['Quantity'][i]*BS_bids['price'][i]*yS[i] for i in range(L_S))
                +sum(FBid_D['Quantity'][i]*FBid_D['price'][i] for i in range(L_FD))*sum(V_D[h,i]for i in range(L_FD) for h in H)-sum(FBid_S['Quantity'][i]*FBid_S['price'][i] for i in range(L_FS))*sum(V_S[h,i]for i in range(L_FS) for h in H)
                
,GRB.MAXIMIZE)

In [114]:
gamma = 100000000000000000

        # (2)
E.addConstrs(-sum((Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0]) * xD[h,j] for j in J ) +sum((Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])*xS[h,j] for j in J) -Pair(1,h,Data_H_D,'DQ')[0] +Pair(1,h,Data_H_SP,'SQ')[0] 
                    +sum(Delta_S[i,h-1]*BS_bids['Quantity'][i]*yS[i] for i in range(L_S))  -sum(Delta_D[i,h-1]*BD_bids['Quantity'][i]*yD[i] for i in range(L_D))
                    +sum(FBid_S['Quantity'][i]*V_S[h,i]for i in range(L_FS))-sum(FBid_D['Quantity'][i]*V_D[h,i]for i in range(L_FD))
                    ==0 for h in H) 
 


E.addConstrs(F_zS[h] == Sm_Res[h-1][0]+sum((Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]) * xS[h,j] for j in J)for h in H)        
E.addConstrs(F_zD[h] == DMax_Res[h-1][0]+sum((Pair(j,h,Data_H_D,'DP')[1]-Pair(j,h,Data_H_D,'DP')[0]) * xD[h,j] for j in J) for h in H)
# (8)

E.addConstrs(-BS_bids['Duration'][j]* BS_bids['price'][j]+ sum(Delta_S[j,h-1]*F_zS[h] for h in H ) <= yS[j]*gamma for j in range(L_S))
# (9)

E.addConstrs(BD_bids['Duration'][j]*BD_bids['price'][j]- sum(Delta_S[j,h-1]*F_zD[h] for h in H) <= yD[j]*gamma for j in range(L_D))
        
    # E.addConstr(if Blockbid['LinkID'][j] == 'NaN': Blockbid['Duration'][j]*Blockbid['price'][j]+ sum(Delta_S[j,h]*F_z[h] for j in J) <= yd[j]*gamma)
        
# (3)(4)
# Demand
E.addConstrs(w_D[h,1]<=xD[h,1] for h in H)
E.addConstrs(xD[h,1] <=1 for h in H)
E.addConstrs(w_D[h,j]<=xD[h,j] for j in range(2,find_largest(HIndex_D)+1) for h in H)
E.addConstrs(xD[h,j]<=w_D[h,j-1] for j in range(2,find_largest(HIndex_D)+1) for h in H)
# Supply
E.addConstrs(w_S[h,1]<=xS[h,1] for h in H)
E.addConstrs(xS[h,1] <=1 for h in H)
E.addConstrs(w_S[h,j]<=xS[h,j] for j in range(2,find_largest(HIndex_S)+1) for h in H)
E.addConstrs(xS[h,j]<=w_S[h,j-1] for j in range(2,find_largest(HIndex_S)+1) for h in H)

E.addConstrs(sum(V_S[h,i] for h in H )<= 1 for i in range(L_FS) )
E.addConstrs(sum(V_D[h,i] for h in H )<= 1 for i in range(L_FD) )

E.addConstrs(F_zS[h]-FBid_S['price'][i] <= gamma*sum(V_S[k,i] for k in H) for i in range(L_FS) for h in H)
E.addConstrs(FBid_D['price'][i]-F_zD[h]<=gamma*sum(V_D[k,i] for k in H) for i in range(L_FD) for h in H)




{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model Update*>,
 (0, 20): <gurobi.Constr *Awaiting Model

In [115]:
E.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 97426 rows, 97352 columns and 325453 nonzeros
Model fingerprint: 0x353d2e4d
Model has 47952 quadratic objective terms
Variable types: 95952 continuous, 1400 integer (1400 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+17]
  Objective range  [1e+02, 2e+07]
  QObjective range [2e-02, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 48604 rows and 47944 columns
Presolve time: 0.20s
Presolved: 48822 rows, 49408 columns, 192715 nonzeros
Presolved model has 47952 quadratic objective terms
Variable types: 47980 continuous, 1428 integer (1428 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   38632   -1.9957344e+08   3.008827e+04 

In [122]:
E.getAttr('x',yD)
E.getAttr('x',F_zD)
E.getAttr('x',V_D)


{(1, 0): 0.0,
 (1, 1): 0.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (1, 5): 0.0,
 (1, 6): 0.0,
 (1, 7): 0.0,
 (1, 8): 0.0,
 (1, 9): 0.0,
 (1, 10): 0.0,
 (1, 11): 0.0,
 (1, 12): 0.0,
 (1, 13): 0.0,
 (1, 14): 0.0,
 (1, 15): 0.0,
 (1, 16): 0.0,
 (1, 17): 0.0,
 (1, 18): 0.0,
 (1, 19): 0.0,
 (1, 20): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (2, 5): 0.0,
 (2, 6): 0.0,
 (2, 7): 0.0,
 (2, 8): 0.0,
 (2, 9): 0.0,
 (2, 10): 0.0,
 (2, 11): 0.0,
 (2, 12): 0.0,
 (2, 13): 0.0,
 (2, 14): 0.0,
 (2, 15): 0.0,
 (2, 16): 0.0,
 (2, 17): 0.0,
 (2, 18): 0.0,
 (2, 19): 0.0,
 (2, 20): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (3, 5): 0.0,
 (3, 6): 0.0,
 (3, 7): 0.0,
 (3, 8): 0.0,
 (3, 9): 0.0,
 (3, 10): 0.0,
 (3, 11): 0.0,
 (3, 12): 0.0,
 (3, 13): 0.0,
 (3, 14): 0.0,
 (3, 15): 0.0,
 (3, 16): 0.0,
 (3, 17): 0.0,
 (3, 18): 0.0,
 (3, 19): 0.0,
 (3, 20): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 0.0,
 (4, 5): 0.0,
 

In [117]:
Res_D = []
Res_S = []
for i in range(1,1000):

    Res_D.append(E.getAttr('x',xD)[(1,i)])
    Res_S.append(E.getAttr('x',xS)[(1,i)])

In [118]:
for i in range(len(Res_D)):
    if Res_D[i] >= 1e-8 and Res_D[i] <= 0.999:
        print(i+1)
for i in range(len(Res_S)):
    if Res_S[i] >= 1e-8 and Res_S[i] <= 0.999:
        print(i+1)

316
317


In [119]:
# def delta(SD):
#     if  SD == 'D':
#         BD_bids  = Blockbid()[1]
#         for i in range (L_D):
#             if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=24:
#                 Delta_D[i,BD_bids['Hour'][i]-1:BD_bids['Hour'][i]+BD_bids['Duration'][i]] = 1
#         return(Delta_D)
#     if  SD == 'S':
#         BS_bids = Blockbid()[0]
#         for i in range (L_S):
#             if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=24:
#                 Delta_S[i,BS_bids['Hour'][i]-1:BS_bids['Hour'][i]+BS_bids['Duration'][i]] = 1
#         return(Delta_S)


BD_bids  = Blockbid()[1]
for i in range (L_D):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=24:
        Delta_D[i,BD_bids['Hour'][i]-1:BD_bids['Hour'][i]+BD_bids['Duration'][i]] = 1

BS_bids = Blockbid()[0]
for i in range (L_S):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=24:
        Delta_S[i,BS_bids['Hour'][i]-1:BS_bids['Hour'][i]+BS_bids['Duration'][i]] = 1


In [120]:
len(F_z)

NameError: name 'F_z' is not defined

In [ ]:
BD_bids  = Blockbid()[1]
BD_bids['Hour'][0]+BD_bids['Duration'][0]

22